In [7]:
import os
from typing import Literal
from time import time
import json
from faithfulness_eval_utils.entailment_eval import evaluate_entailment

In [3]:
with open("../sample_complex.json") as fin:
    data = json.load(fin)

evaluate_entailment(
    data,
    result_dir="output/roberta_mnli",
    tokenizer_path="roberta-large-mnli",
    model_path="roberta-large-mnli",
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved 5 items to `output/roberta_mnli/results.json`
Averages written to `output/roberta_mnli/stats.json`


([{'statement': "Shakespeare wrote 'Romeo and Juliet'.",
   'label': 'SUPPORTED',
   'evidences': ["William Shakespeare is credited as the author of the play 'Romeo and Juliet'.",
    "'Romeo and Juliet' was first published in 1597 and is attributed to Shakespeare."],
   'explanation': "Historical records and publication attributions confirm that Shakespeare wrote 'Romeo and Juliet'.",
   'e2x_entail_prob': 0.38058656454086304,
   'x2e_entail_prob': 0.6615111231803894},
  {'statement': 'Eating carrots improves eyesight.',
   'label': 'NOT_SUPPORTED',
   'evidences': ['Carrots contain beta-carotene, which the body converts to vitamin A, a nutrient important for vision.',
    'No clinical evidence shows that eating carrots prevents or cures vision problems beyond normal health benefits.'],
   'explanation': 'While carrots provide vitamin A, there is no evidence they improve eyesight beyond maintaining normal vision.',
   'e2x_entail_prob': 0.9374357461929321,
   'x2e_entail_prob': 0.3243

In [9]:
SKIP_EXISTING_RESULTS: Literal["auto", "ask"] = "auto"

data_path = os.path.normpath("../inference_outputs/converted_outputs/")
model_setup_dirs = os.listdir(data_path)

t1 = time()

for model_setup_dir in model_setup_dirs:
    if not os.path.isdir(os.path.join(data_path, model_setup_dir)):
        continue

    print("\n\n+++++++++++++++++++++++++++++++++++++++++++")
    print(f"Processing model setup: {model_setup_dir}")

    dataset_files = os.listdir(os.path.join(data_path, model_setup_dir))
    if not dataset_files:
        print(f"No files found in {model_setup_dir}...")
        raise ValueError()

    for dataset_file in dataset_files:
        if not dataset_file.endswith(".json"):
            continue

        t2 = time()
        dataset_path = os.path.join(data_path, model_setup_dir, dataset_file)
        result_dir = os.path.join("output", "entailment", model_setup_dir)
        stats_output_filename = f"{dataset_file.replace('.json', '.stats.json')}"
        results_output_filename = f"{dataset_file.replace('.json', '.results.json')}"

        if os.path.exists(
            os.path.join(result_dir, stats_output_filename)
        ) or os.path.exists(os.path.join(result_dir, results_output_filename)):
            print(
                f"Either {stats_output_filename} or {results_output_filename} already exists in {result_dir}."
            )
            if SKIP_EXISTING_RESULTS == "ask":
                if (
                    input("Do you want to recreate these files? (y/n): ")
                    .strip()
                    .lower()
                    != "y"
                ):
                    print("Skipping these files...")
                    continue
            elif SKIP_EXISTING_RESULTS == "auto":
                print(
                    f"Skipping {dataset_path} as results already exist in {result_dir}. Please check manually if results are valid."
                )
                continue

        print(f"\nEvaluating {dataset_path}...")
        with open(dataset_path, encoding="utf-8") as fin:
            data = json.load(fin)

        evaluate_entailment(
            data=data,
            stats_filename=stats_output_filename,
            results_filename=results_output_filename,
            result_dir=result_dir,
            tokenizer_path="roberta-large-mnli",
            model_path="roberta-large-mnli",
        )
        t3 = time()
        print(
            f"Finished evaluating {dataset_path}. Time elapsed since start: {(t3 - t1) / 60:.2f} minutes, since last file: {(t3 - t2) / 60:.2f} minutes."
        )




+++++++++++++++++++++++++++++++++++++++++++
Processing model setup: deepseek_r1_32b_cot

Evaluating ..\inference_outputs\converted_outputs\deepseek_r1_32b_cot\covid_fact_depseek_r1_cot.json...


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved 1017 items to `output\entailment\deepseek_r1_32b_cot/covid_fact_depseek_r1_cot.results.json`
Averages written to `output\entailment\deepseek_r1_32b_cot/covid_fact_depseek_r1_cot.stats.json`
Finished evaluating ..\inference_outputs\converted_outputs\deepseek_r1_32b_cot\covid_fact_depseek_r1_cot.json. Time elapsed since start: 17.22 minutes, since last file: 17.22 minutes.

Evaluating ..\inference_outputs\converted_outputs\deepseek_r1_32b_cot\hover_train_depseek-r1-cot.json...


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved 2076 items to `output\entailment\deepseek_r1_32b_cot/hover_train_depseek-r1-cot.results.json`
Averages written to `output\entailment\deepseek_r1_32b_cot/hover_train_depseek-r1-cot.stats.json`
Finished evaluating ..\inference_outputs\converted_outputs\deepseek_r1_32b_cot\hover_train_depseek-r1-cot.json. Time elapsed since start: 61.69 minutes, since last file: 44.47 minutes.

Evaluating ..\inference_outputs\converted_outputs\deepseek_r1_32b_cot\politi_hop_depseek_r1_cot.json...


...

Saved 1162 items to `output\entailment\mistral__7b_cot/covid_fact_mistral_7b_cot.results.json`
Averages written to `output\entailment\mistral__7b_cot/covid_fact_mistral_7b_cot.stats.json`
Finished evaluating ..\inference_outputs\converted_outputs\mistral__7b_cot\covid_fact_mistral_7b_cot.json. Time elapsed since start: 337.88 minutes, since last file: 24.06 minutes.

Evaluating ..\inference_outputs\converted_outputs\mistral__7b_cot\hover_train_mistral_7b_cot.json...


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved 2558 items to `output\entailment\mistral__7b_cot/hover_train_mistral_7b_cot.results.json`
Averages written to `output\entailment\mistral__7b_cot/hover_train_mistral_7b_cot.stats.json`
Finished evaluating ..\inference_outputs\converted_outputs\mistral__7b_cot\hover_train_mistral_7b_cot.json. Time elapsed since start: 401.87 minutes, since last file: 63.99 minutes.

Evaluating ..\inference_outputs\converted_outputs\mistral__7b_cot\politi_hop_mistral_7b_cot.json...


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved 497 items to `output\entailment\mistral__7b_cot/politi_hop_mistral_7b_cot.results.json`
Averages written to `output\entailment\mistral__7b_cot/politi_hop_mistral_7b_cot.stats.json`
Finished evaluating ..\inference_outputs\converted_outputs\mistral__7b_cot\politi_hop_mistral_7b_cot.json. Time elapsed since start: 415.74 minutes, since last file: 13.87 minutes.
